### Configuration

In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr

import mne
from mne.time_frequency import tfr_array_morlet
from scipy.stats import zscore
from tqdm import tqdm

from utils__helpers_macro import hilbert_powerphase, hilbert_envelope
import utils__config

In [2]:

os.chdir(utils__config.working_directory)
os.getcwd()

'g:\\My Drive\\Residency\\Research\\Lab - Damisah\\Project - Sleep'

### Parameters

In [3]:
fif_path = 'Cache/Subject01/S01_Feb02_256hz.fif'
tfr_path = 'Cache/Subject01/S01_tfr_30s_epochs.csv'

# fif_path = 'Cache/Subject02/Apr26/S02_Apr26_256hz.fif'
# tfr_path = 'Cache/Subject02/Apr26/S02_tfr_30s_epochs.csv'

# fif_path = 'Cache/Subject02/Apr27/S02_Apr27_256hz.fif'
# tfr_path = 'Cache/Subject02/Apr27/S02_tfr_30s_epochs.csv'

In [4]:
sampling_freq = 256
resample_frequency = 128 # frequency to resample to prior to Morlet (reduces memory usage)
tfr_decimation = 1 # decimation by Morlet; reduces memory usage but removes the ability to keep true time!
mean_bin_division = (resample_frequency / tfr_decimation) * 30 # division factor to bin samples into mean
rolling_mean_samples = 3 # number of samples over which to calculate rolling mean

### Morlet Transform

In [5]:
# Load Data
raw = mne.io.read_raw_fif(fif_path, preload = True, verbose = None)

# # Keep channels that have detected sleep events 
# # (also include the central sleep staging electrode)
# events = pd.read_csv(events_path)
# event_channels = events['channel'].unique().tolist()
# keep_channels = event_channels + ['C4']
# raw.pick_channels(ch_names = keep_channels)

Opening raw data file Cache/Subject01/S01_Feb02_256hz.fif...


C:\Users\Layton\AppData\Local\Temp\ipykernel_21356\4279266254.py:2: RuntimeWarning: This filename (Cache/Subject01/S01_Feb02_256hz.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fif_path, preload = True, verbose = None)


Isotrak not found
    Range : 0 ... 1843573 =      0.000 ...  7201.457 secs
Ready.
Reading 0 ... 1843573  =      0.000 ...  7201.457 secs...


<Raw | S01_Feb02_256hz.fif, 52 x 1843574 (7201.5 s), ~731.5 MB, data loaded>

In [6]:
# Decimate to reduce memory usage
raw.resample(resample_frequency)

# Save timestamps for later
timestamps = raw.times

if tfr_decimation > 1:
    timestamps = timestamps[::tfr_decimation]

# Format Data for tfr_array_morlet()
ts_array = raw.get_data(units = dict(seeg = 'uV', eeg = 'uV'))
ts_array = ts_array[np.newaxis, :, :]

In [7]:
freqs = np.arange(1, 26, 1)

# Create time-frequency representation
# using the Morlet Wavelet transform:
tfr = tfr_array_morlet(ts_array, 
                       sfreq = raw.info['sfreq'],
                       freqs = freqs, 
                       n_cycles = 6.0,
                       zero_mean = False, 
                       use_fft = True, 
                       decim = tfr_decimation, 
                       output = 'power', 
                       n_jobs = -1, 
                       verbose = None)

# Remove the dummy dimension (that was required
# due to formatting expectations of MNE Morlet):
tfr = np.squeeze(tfr)

# Convert to Xarray as an intermediate step in
# getting data into Pandas long (2d) format:
tfr = xr.DataArray(tfr,
                   dims = ('channel', 'frequency', 'seconds'),
                   coords = {'channel' : raw.ch_names,
                             'frequency' : freqs,
                             'seconds' : timestamps})

tfr = tfr.to_dataframe(name = 'power').reset_index()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   36.0s
[Parallel(n_jobs=-1)]: Done  48 out of  52 | elapsed:  1.7min remaining:    8.3s
[Parallel(n_jobs=-1)]: Done  52 out of  52 | elapsed:  1.7min finished


### Time Bin for Convenience

In [8]:
# Average 1 second values into 30s epoch values
# (bin stop value is arbitrarily large, 1mil seconds is 277 hrs)
# (alternative to the next two optional steps)
bin_list = np.arange(0, 1000000, 30)

tfr['epoch'] = pd.cut(tfr['seconds'], bins = bin_list, labels = False)
tfr = tfr.groupby(['channel', 'frequency', 'epoch']).mean('power')
tfr = tfr.reset_index()[['channel', 'frequency', 'epoch', 'power']]

### Log Normalize by Frequency

In [9]:
# Channel-wise and frequency-wise normalization, 
# since each channel (each one has a different
# baseline power magnitude) and frequency (1/f):
tfr['meanpower'] = tfr.groupby(['channel', 'frequency'])['power'].transform('mean')
tfr['log_meanpower'] = 10 * np.log10(tfr['meanpower'])

tfr['logpower'] = 10 * np.log10(tfr['power'])
tfr['logpower_mean'] = tfr.groupby(['channel', 'frequency'])['logpower'].transform('mean')

tfr['logmpower_freq'] = tfr['logpower'] - tfr['log_meanpower']
tfr['logpower_freq'] = tfr['logpower'] - tfr['logpower_mean']

tfr.drop(columns = ['meanpower', 'log_meanpower', 'logpower_mean'], inplace = True)

# Now calculate frequency-wise zscores from the log(power)
tfr['lmpf_zscore'] = tfr.groupby(['channel', 'frequency'])['logmpower_freq'].transform(zscore)
tfr['lpf_zscore'] = tfr.groupby(['channel', 'frequency'])['logpower_freq'].transform(zscore)

### Mean, Smooth, and/or Decimate

Note how this processing is different than the Morlet TFR computation for C4 in the `Transitions` script.

In [10]:
# # Group by chan, freq, and time bin; define the time bin using floor
# # division of seconds by your desired decimation factor (optional step)
# tfr['time_bin'] = tfr.groupby(['channel', 'frequency']).cumcount() + 1
# tfr['time_bin'] = tfr['time_bin'] // mean_bin_division
# tfr = tfr.groupby(['channel', 'frequency', 'time_bin']).mean()
# tfr = tfr.reset_index()

# Rolling mean to smooth TFR (optional step)
saved_epochs = tfr['epoch']

tfr = tfr.groupby(['channel', 'frequency']).rolling(window = rolling_mean_samples, 
                                                    min_periods = 1, 
                                                    center = True, 
                                                    win_type = 'gaussian').mean()
tfr = tfr.reset_index()
tfr.drop(columns = ['level_2'], inplace = True)

tfr['epoch'] = saved_epochs

C:\Users\Layton\AppData\Local\Temp\ipykernel_21356\2576674005.py:11: FutureWarning: Passing additional kwargs to RollingGroupby.mean has no impact on the result and is deprecated. This will raise a TypeError in a future version of pandas.
  tfr = tfr.groupby(['channel', 'frequency']).rolling(window = rolling_mean_samples,


In [11]:
tfr.to_csv(tfr_path, index = False)

In [12]:

print(pd.__version__)

1.5.3
